In [1]:
!nvidia-smi

Fri Dec 16 21:26:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    37W / 300W |      0MiB / 16160MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
class my_args:
    def __init__(self, generator_name, dataset_name, dataset_percent, num_cands, scorer_path):
        self.generator_name = generator_name
        self.dataset_name = dataset_name
        self.num_cands = dataset_percent
        self.dataset_percent = num_cands
        self.scorer_path = scorer_path
args = my_args('tuned_t5_model','clean_covid.csv',8,10, 'cache/22-12-16-0/scorer.bin') # 10 stands for 10% of dataset are used

In [3]:
!python gen_candidate.py --generator_name {args.generator_name} --dataset_name {args.dataset_name} --dataset_percent {args.dataset_percent} --num_cands {args.num_cands}

start tokenize dataset--------------------
100%|█████████████████████████████████████████████| 9/9 [00:03<00:00,  2.54ba/s]
start generate training candidates!--------------------------------------------------
start generate validation candidates!--------------------------------------------------
start generate testing candidates!--------------------------------------------------
start saving candidate summariesch_index: 255, last epoch time: 3 sec


In [4]:
# create folder for processed dataset
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/train')
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/test')
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/val')

# process candidates
print('start processing val candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/val --split 'val' --cand_num {args.num_cands}
print('start processing train candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/train --split 'train' --cand_num {args.num_cands}
print('start processing test candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/test --split 'test' --cand_num {args.num_cands}

start processing val candidates for training
finish
start processing train candidates for training
finish
start processing test candidates for training
finish


In [6]:
#make necessary dirs for training
if not os.path.exists('cache'):
    os.makedirs('cache')

In [7]:
# start training scorer
training_data_path = f'candidates/{args.generator_name}_{args.num_cands}_processed'
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path}
# default training for 5 epoch

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Namespace(accumulate_step=12, batch_size=1, cand_weight=1, cuda=True, dataset='candidates/tuned_t5_model_8_processed', datatype='diverse', encode_mode=None, epoch=5, evaluate=False, gold_margin=0, gold_weight=1, gpuid=[0], grad_norm=0, log=True, margin=0.01, max_len=120, max_lr=0.002, max_num=16,

id: 0
similarity: tensor([[0.9521, 0.9544, 0.9502, 0.9535, 0.9499, 0.9782, 0.9704, 0.9552]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9497], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 100, avg loss: 0.307845
learning rate: 0.000000

id: 0
similarity: tensor([[0.9914, 0.9928, 0.9911, 0.9943, 0.9586, 0.9920, 0.9936, 0.9938]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9919], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 200, avg loss: 0.296469
learning rate: 0.000000

id: 0
similarity: tensor([[0.9922, 0.9946, 0.9929, 0.9927, 0.9912, 0.9949, 0.9949, 0.9916]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9944], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 300, avg loss: 0.287116
learning rate: 0.000001

id: 0
similarity: tensor([[0.9715, 0.9913, 0.9896, 0.9923, 0.9848, 0.9938, 0.9933, 0.9935]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold 

id: 0
similarity: tensor([[0.8710, 0.9131, 0.8772, 0.9134, 0.9047, 0.8995, 0.8817, 0.8781]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9308], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 2800, avg loss: 0.257163
learning rate: 0.000006

id: 0
similarity: tensor([[0.9091, 0.9187, 0.9063, 0.9233, 0.9110, 0.9220, 0.9296, 0.9191]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9043], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 2900, avg loss: 0.257848
learning rate: 0.000006

id: 0
similarity: tensor([[0.8475, 0.8538, 0.8587, 0.8664, 0.8293, 0.8265, 0.8757, 0.8721]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8709], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 3000, avg loss: 0.253773
learning rate: 0.000006

test similarity: [0.8503989  0.8658348  0.8396891  0.8438417  0.8391264  0.86114246
 0.838391   0.8626852 ]
test similarity: [0.89342314 0.893556

id: 0
similarity: tensor([[0.9097, 0.9057, 0.9026, 0.9155, 0.9157, 0.8872, 0.9090, 0.9140]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9339], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 1, batch: 5400, avg loss: 0.245956
learning rate: 0.000011

id: 0
similarity: tensor([[0.8582, 0.9055, 0.8853, 0.9103, 0.9214, 0.8716, 0.9142, 0.8965]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9032], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 2, batch: 100, avg loss: 0.240689
learning rate: 0.000011

id: 0
similarity: tensor([[0.8808, 0.8453, 0.8521, 0.8415, 0.8206, 0.8296, 0.8380, 0.8591]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8869], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 2, batch: 200, avg loss: 0.245097
learning rate: 0.000011

id: 0
similarity: tensor([[0.8967, 0.9111, 0.9012, 0.8865, 0.8702, 0.8692, 0.8693, 0.8563]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold

rouge-1: 0.12591361526456651, rouge-2: 0.009195364011608642, rouge-L: 0.10659754237261522
best - epoch: 1, batch: 2567
val rouge: 0.080569
id: 0
similarity: tensor([[0.8773, 0.8981, 0.8885, 0.9186, 0.8835, 0.9274, 0.8664, 0.8688]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8865], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 2, batch: 2600, avg loss: 0.243026
learning rate: 0.000016

id: 0
similarity: tensor([[0.9450, 0.9473, 0.9601, 0.9622, 0.9537, 0.9526, 0.9391, 0.9355]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9189], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 2, batch: 2700, avg loss: 0.243062
learning rate: 0.000016

id: 0
similarity: tensor([[0.9261, 0.9288, 0.9194, 0.9284, 0.9228, 0.9419, 0.9324, 0.9069]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9422], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 2, batch: 2800, avg loss: 0.232566
learning rate: 0.000016

id: 0


...

id: 0
similarity: tensor([[0.8865, 0.8227, 0.8666, 0.8267, 0.8315, 0.8407, 0.8164, 0.7872]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8565], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 3, batch: 5100, avg loss: 0.224480
learning rate: 0.000016

test similarity: [0.826529   0.8376783  0.85120773 0.83435357 0.8307314  0.87661964
 0.8652258  0.8643993 ]
test similarity: [0.7970879  0.8002355  0.8025395  0.7646878  0.7930598  0.8017472
 0.78798497 0.7757331 ]
rouge-1: 0.12594814511581087, rouge-2: 0.009266855880162664, rouge-L: 0.10622144715369147
val rouge: 0.080479
id: 0
similarity: tensor([[0.8831, 0.8734, 0.8664, 0.8588, 0.8650, 0.8685, 0.8567, 0.8573]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8349], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 3, batch: 5200, avg loss: 0.216264
learning rate: 0.000016

id: 0
similarity: tensor([[0.9092, 0.9130, 0.9031, 0.8673, 0.8968, 0.9190, 0.8212, 0.8118]],
       devic

id: 0
similarity: tensor([[0.7777, 0.7129, 0.7307, 0.8249, 0.7387, 0.6909, 0.7347, 0.6945]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8872], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 2300, avg loss: 0.197744
learning rate: 0.000015

id: 0
similarity: tensor([[0.8350, 0.8398, 0.8125, 0.8304, 0.8229, 0.7761, 0.7726, 0.7443]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8071], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 2400, avg loss: 0.190969
learning rate: 0.000015

id: 0
similarity: tensor([[0.8120, 0.7780, 0.8213, 0.8077, 0.8073, 0.7889, 0.7802, 0.8021]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8231], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 2500, avg loss: 0.198451
learning rate: 0.000015

id: 0
similarity: tensor([[0.8364, 0.7963, 0.7873, 0.8122, 0.7502, 0.7813, 0.8227, 0.7608]],
       device='cuda:0', grad_fn=<SliceBackward0>)
go

id: 0
similarity: tensor([[0.8833, 0.8745, 0.8302, 0.8373, 0.8679, 0.8714, 0.8082, 0.8377]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8376], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 4900, avg loss: 0.190552
learning rate: 0.000014

id: 0
similarity: tensor([[0.8820, 0.8909, 0.8720, 0.8614, 0.8382, 0.8426, 0.8259, 0.8301]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8713], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 5000, avg loss: 0.190197
learning rate: 0.000014

id: 0
similarity: tensor([[0.8076, 0.8112, 0.8051, 0.8102, 0.7937, 0.7961, 0.7994, 0.8087]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.7892], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 4, batch: 5100, avg loss: 0.201907
learning rate: 0.000014

id: 0
similarity: tensor([[0.8636, 0.8706, 0.8527, 0.8615, 0.8419, 0.8181, 0.8406, 0.8212]],
       device='cuda:0', grad_fn=<SliceBackward0>)
go

id: 0
similarity: tensor([[0.8265, 0.8218, 0.8453, 0.7527, 0.7838, 0.7947, 0.8086, 0.7579]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.7837], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 5, batch: 2200, avg loss: 0.165918
learning rate: 0.000013

test similarity: [0.7133251  0.7331007  0.7203011  0.7171203  0.7143634  0.7459508
 0.7543956  0.73979896]
test similarity: [0.62162316 0.62182647 0.64826864 0.59522086 0.60454744 0.6551249
 0.62486327 0.6026355 ]
rouge-1: 0.12640537229206752, rouge-2: 0.00978820931951384, rouge-L: 0.10632433379013147
val rouge: 0.080839
id: 0
similarity: tensor([[0.8136, 0.8867, 0.8252, 0.7593, 0.8299, 0.8142, 0.7948, 0.8066]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.7968], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 5, batch: 2300, avg loss: 0.165665
learning rate: 0.000013

id: 0
similarity: tensor([[0.8171, 0.8113, 0.8035, 0.7951, 0.7976, 0.8426, 0.7787, 0.8723]],
       device=

id: 0
similarity: tensor([[0.8563, 0.8423, 0.8674, 0.9035, 0.8444, 0.9400, 0.8370, 0.8588]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.9055], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 5, batch: 4800, avg loss: 0.164439
learning rate: 0.000012

id: 0
similarity: tensor([[0.8641, 0.8475, 0.7582, 0.8146, 0.7704, 0.8438, 0.7512, 0.7691]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8167], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 5, batch: 4900, avg loss: 0.168769
learning rate: 0.000012

id: 0
similarity: tensor([[0.7772, 0.7985, 0.8068, 0.8044, 0.7876, 0.7724, 0.7515, 0.7563]],
       device='cuda:0', grad_fn=<SliceBackward0>)
gold similarity: tensor([0.8280], device='cuda:0', grad_fn=<DivBackward0>)
epoch: 5, batch: 5000, avg loss: 0.165640
learning rate: 0.000012

id: 0
similarity: tensor([[0.8495, 0.8730, 0.8405, 0.8334, 0.7885, 0.8468, 0.8521, 0.8628]],
       device='cuda:0', grad_fn=<SliceBackward0>)
go

In [4]:
# start evaluating scorer just trained
!python evaluate_model.py --generator_name {args.generator_name} --dataset_name {args.dataset_name} --scorer_path {args.scorer_path} --dataset_percent 1



start tokenize data
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.76ba/s]
There are 814 samples in test set!
start loading scorer model
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
start evaluating, total_num of samples is:814
Truncation was not explicitly activated but `max_length` is provided a specific value, p